# Practice with the Python ATProtoSDK
This ipython notebook will walk you through the basics of working with the
ATProto python sdk. The content here heavily draws on [these examples](https://github.com/MarshalX/atproto/tree/main/examples)

In [1]:
from atproto import Client
from dotenv import load_dotenv
import os
import pprint


load_dotenv(override=True)
USERNAME = os.getenv("USERNAME")
PW = os.getenv("PW")

## Logging into your account

In [ ]:
client = Client()
profile = client.login(USERNAME, PW)
pprint.pprint(profile.__dict__)
# https://bsky.social/xrpc/com.atproto.sync.bafkreihijcc5i4pjtect2ou6wzv2b4jnc657f24iz6e3zl7k5amhh3bmh4
https://bsky.social/xrpc/com.atproto.repo.getBlob?did=did:plc:yzpplgm5kftdgpf2wsnrbgdn&cid=bafkreihijcc5i4pjtect2ou6wzv2b4jnc657f24iz6e3zl7k5amhh3bmh4

{'associated': ProfileAssociated(chat=None, feedgens=0, labeler=True, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'),
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:yzpplgm5kftdgpf2wsnrbgdn/bafkreih3fpryxoepb44fzyr3sfn32fr7fqqka4kle6h4not7jlwtdvzghe@jpeg',
 'banner': None,
 'created_at': '2025-02-13T17:12:12.845Z',
 'description': None,
 'did': 'did:plc:yzpplgm5kftdgpf2wsnrbgdn',
 'display_name': '',
 'followers_count': 0,
 'follows_count': 1,
 'handle': 'trustylabeler.bsky.social',
 'indexed_at': '2025-02-13T17:12:12.845Z',
 'joined_via_starter_pack': None,
 'labels': [],
 'pinned_post': None,
 'posts_count': 0,
 'py_type': 'app.bsky.actor.defs#profileViewDetailed',
 'viewer': ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState')}


## Working with posts

In [3]:
def post_from_url(client: Client, url: str):
    """
    Retrieve a Bluesky post from its URL
    """
    parts = url.split("/")
    rkey = parts[-1]
    handle = parts[-3]
    return client.get_post(rkey, handle)

post = post_from_url(client, "https://bsky.app/profile/labeler-test.bsky.social/post/3lktj7ewxxv2q")
pprint.pprint(post.value.__dict__)

{'created_at': '2025-03-20T20:14:57.103160+00:00',
 'embed': Main(images=[Image(alt='dog', image=BlobRef(mime_type='image/jpeg', size=169278, ref=IpldLink(link='bafkreibahplioamouecglrcqnshcxzdrwawdtwl5h676d2l7k7xbbti3pa'), py_type='blob'), aspect_ratio=None, py_type='app.bsky.embed.images#image')], py_type='app.bsky.embed.images'),
 'entities': None,
 'facets': None,
 'labels': None,
 'langs': ['en'],
 'py_type': 'app.bsky.feed.post',
 'reply': None,
 'tags': None,
 'text': 'check out this dog!'}


In [4]:
post.value.text

'check out this dog!'

In [5]:
# https://github.com/MarshalX/atproto/blob/main/examples/profile_posts.py
prof_feed = client.get_author_feed(actor="weratedogs.com")
for i, feed_view in enumerate(prof_feed.feed[:10]):
    print(f"Post {i}:", feed_view.post.record.text)

post = prof_feed.feed[0].post
likes_resp = client.get_likes(post.uri, post.cid, limit=10)
print("Likes:", [like.actor.handle for like in likes_resp.likes])

post_thread_resp = client.get_post_thread(post.uri)
print([rep.post.record.text for rep in post_thread_resp.thread.replies[:10]])

Post 0: This is Autumn. She was born without one paw. Gets along just fine with three, but still sometimes uses a prosthetic leg when she wants to blend in. 13/10
Post 1: We have a tough update on Marceline 💔 

The mass removed from her jaw was recently confirmed to be a rare, aggressive bone tumor. This means that at 2 years old, Marceline’s time is limited. It may be months or years. Thanks to you, she is able to live every day with joy. That means everything ❤️‍🩹
Post 2: ‪sweet angel 🥺❤️‬
Post 3: Bah! We only rate dogs. This is a sheep. Please look up what a dog is, and only send us that. Thank you… 12/10 (IG: onathestandardpoodle)
Post 4: another great pick
Post 5: This is Max. He has our sticker on his window. You are now legally obligated to tell your dog Max said hi. (we're serious) Get the sticker below!

www.weratedogs.com?utm_source=b...
Post 6: This is Tenney. She has a case of the zoomies. And also the agility of a running back. 13/10
Post 7: This is Samson. He thought he w

## Followers/following

How might you use this information to investigate/mitigate a harm?

In [7]:

follower_resp = client.get_followers("weratedogs.com", limit=10)
following_resp = client.get_follows("weratedogs.com", limit=10)
print("Followers:", [follower.handle for follower in follower_resp.followers])
print("Following:", [follow.handle for follow in following_resp.follows])



Followers: ['kelly3010.bsky.social', 'homaksu.bsky.social', 'jennifersamule.bsky.social', 'walkerbn.bsky.social', 'cuneyterdem0.bsky.social', 'shinddha.bsky.social', 'yooperann.bsky.social', 'tor37.bsky.social', 'windowtothesoull.bsky.social', 'twocakesup.bsky.social']
Following: ['15outof10.org']


## Exercise: Compute average dog ratings
The WeRateDogs account includes ratings out of 10 within some of its posts.
Write a script that computes the average rating (out of 10) for the 100 most
recent posts from this account. (note that not every post will have a rating)

In [9]:
import re

In [10]:
def extract_score(text):
    # This pattern looks for:
    # \d+ - one or more digits
    # \s* - optional whitespace
    # / - literal forward slash
    # \s* - optional whitespace
    # 10 - literal "10"
    pattern = r'(\d+)\s*/\s*10'
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    return None

In [13]:
def compute_avg_dog_rating(num_posts):
    # TODO: complete
    total_score = 0
    num_scores = 0
    for i, feed_view in enumerate(prof_feed.feed[:num_posts]):
        score = extract_score(feed_view.post.record.text)
        if score is not None:
            # print(f"Found score: {score}/10")
            total_score += score
            num_scores += 1
        # else:
            # print("No score found in the format X/10")
    return total_score / num_scores if num_scores > 0 else 0

print("The average rating is:", compute_avg_dog_rating(100))

The average rating is: 13.0


## Exercise: Dog names
Collect the names of dogs within the latest 100 posts and print them to the
console. Hint: see if you can identify a pattern in the posts.

In [15]:
def extract_names(text):
    # This pattern looks for:
    # (?<![\.\?\!]\s) - negative lookbehind for period/question mark/exclamation followed by whitespace
    # (?<!\A) - negative lookbehind for start of string
    # \b[A-Z][a-zA-Z]*\b - word boundary, capital letter, followed by any letters, word boundary
    pattern = r'(?<![\.\?\!]\s)(?<!\A)\b[A-Z][a-zA-Z]*\b'
    
    matches = re.finditer(pattern, text)
    return [match.group() for match in matches]

In [16]:
def collect_dog_names(num_posts):
    # TODO: complete
    all_names = []
    for i, feed_view in enumerate(prof_feed.feed[:num_posts]):
        names = extract_names(feed_view.post.record.text)
        if names:
            all_names.extend(names)
    return all_names

print("Here are the dog_names:", collect_dog_names(100))

Here are the dog_names: ['George', 'SeniorPupSaturday', 'Fiyero', 'Dasia', 'The', 'Farmers', 'Dog', 'Top', 'Dogs', 'Dill', 'GIANT', 'Roger', 'Roger', 'ER', 'Panda', 'We', 'Panda', 'Panda', 'I', 'Ellie', 'Flipper', 'ALL', 'Good', 'Lucie', 'Muamba', 'Ollie', 'Heckles', 'ONLY', 'April', 'Top', 'WORST', 'Dogs', 'Joey', 'Bambi', 'Top', 'Dogs', 'March', 'Pippa', 'SeniorPupSaturday', 'Thank', 'Butterfly', 'Butterfly', 'Top', 'Dogs', 'Leon', 'Hubie', 'Fig', 'Oreo', 'FBI', 'Choco', 'We', 'ADOPTED', 'Choco', 'Rex']


## Exercise: Soliciting donations
Some posts from the WeRateDogs account ask for donations -- usually for
covering medical costs for the featured dogs. Within the latest 100 posts, print
the text content of those that fall into this category

In [ ]:
def donation_posts(num_posts):
    # TODO: complete
    return []

for post_text in donation_posts(100):
    print(post_text)